<a href="https://colab.research.google.com/github/george-zakharov/ml-random/blob/master/lips_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libs

In [9]:
%tensorflow_version 2.x

In [10]:
import tensorflow
print(tensorflow.__version__)

2.3.0


In [35]:
import os
import shutil
import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from google.colab import drive
from google.colab import data_table

# from google.colab import files
from tensorflow.keras.preprocessing import image
# from tensorflow.keras import utils
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Model
# from keras.layers import Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization
# from keras import backend as K
# from keras.optimizers import Adam
# import matplotlib.pyplot as plt

# Load data

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [45]:
# Get raw data
!cp '/content/drive/My Drive/Colab Notebooks/segmentation_test/lips.zip' lips.zip

In [4]:
# Get pretrained values
!cp '/content/drive/My Drive/Colab Notebooks/segmentation_test/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5' vgg16.h5

In [ ]:
# Unzip it
!unzip lips.zip

In [6]:
# Delete archive
!rm lips.zip

# Check the mappings

In [7]:
mappings = pd.read_csv('set-lipstick-original/list.csv')

In [37]:
mappings.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,mask
0,image00000001.jpg,1280,720,Lips,661,394,776,444,mask00000001.png
1,image00000002.jpg,1280,720,Lips,557,336,682,392,mask00000002.png
2,image00000003.jpg,1280,720,Lips,553,369,684,427,mask00000003.png
3,image00000004.jpg,1280,720,Lips,555,351,681,408,mask00000004.png
4,image00000005.jpg,1280,720,Lips,555,351,680,407,mask00000005.png


# Data processing

In [13]:
# Prepare dirs for files
!mkdir DIR_X_TRAIN
!mkdir DIR_X_TEST
!mkdir DIR_Y_TRAIN
!mkdir DIR_Y_TEST

In [38]:
# Get first 5000 pics for training
train_df = mappings.loc[0:499]

In [39]:
# Get test pics
test_df = mappings.loc[501:600]

In [31]:
# Copy train pics
for index, row in train_df.iterrows():
    shutil.copy('/content/set-lipstick-original/720p/' + row['filename'], '/content/DIR_X_TRAIN/' + row['filename'])
    shutil.copy('/content/set-lipstick-original/mask/' + row['mask'], '/content/DIR_Y_TRAIN/' + row['mask'])

In [40]:
# Copy test pics
for index, row in test_df.iterrows():
    shutil.copy('/content/set-lipstick-original/720p/' + row['filename'], '/content/DIR_X_TEST/' + row['filename'])
    shutil.copy('/content/set-lipstick-original/mask/' + row['mask'], '/content/DIR_Y_TEST/' + row['mask'])

In [43]:
# Let's drop source images
!rm -rf /content/set-lipstick-original/

In [44]:
# Get all paths
x_test_files = glob.glob('/content/DIR_X_TEST/*')
x_train_files = glob.glob('/content/DIR_X_TRAIN/*')
y_test_files = glob.glob('/content/DIR_Y_TEST/*')
y_train_files = glob.glob('/content/DIR_Y_TRAIN/*')